# Example Brick Model for process
Example brick model will be 2 AHUs, 60 terminal units.
Half are for perimeter zones vavs with reheat, half are for core zones and are cooling only 

10 core zone terminal units are for mechanical rooms, networking closets, and electrical rooms that must be served 24/7

In [1]:
from rdflib import Namespace, URIRef, RDFS
from buildingmotif import BuildingMOTIF
from buildingmotif.dataclasses import Model, Library
from buildingmotif.namespaces import BMOTIF, BRICK
from buildingmotif.model_builder import TemplateBuilderContext

In [2]:
BLDG = Namespace("urn:example#")
bm = BuildingMOTIF("sqlite://", "topquadrant")
bldg = Model.create("urn:example#")
bldg.graph.bind('',BLDG)
brick = Library.load(ontology_graph="Brick.ttl")

2025-07-03 14:21:51,442 | root |  WARNING: An ontology could not resolve a dependency on http://qudt.org/2.1/vocab/unit (Name: http://qudt.org/2.1/vocab/unit). Check this is loaded into BuildingMOTIF
2025-07-03 14:21:51,443 | root |  WARNING: An ontology could not resolve a dependency on https://w3id.org/rec/recimports (Name: https://w3id.org/rec/recimports). Check this is loaded into BuildingMOTIF
2025-07-03 14:21:51,443 | root |  WARNING: An ontology could not resolve a dependency on http://data.ashrae.org/bacnet/2020 (Name: http://data.ashrae.org/bacnet/2020). Check this is loaded into BuildingMOTIF
2025-07-03 14:21:51,444 | root |  WARNING: An ontology could not resolve a dependency on http://qudt.org/2.1/schema/shacl/overlay/qudt (Name: http://qudt.org/2.1/schema/shacl/overlay/qudt). Check this is loaded into BuildingMOTIF
2025-07-03 14:21:51,444 | root |  WARNING: An ontology could not resolve a dependency on http://qudt.org/2.1/vocab/sou (Name: http://qudt.org/2.1/vocab/sou). Ch

In [3]:
brick_tmpl = Library.load(directory='brick-templates')

In [4]:
# start with the tank. This instantiates the template with 'my_tank' as the name
# You can assign other parameters here as well. If the value you give to a parameter
# is a string (e.g. 'my_tank') then the builder will automatically make a URL with the BLDG (see cell above) as the namespace
# ahu = ctx['multiple-zone-ahu'](name='ahu1')
# # now create an instance of the water flow property
# flow_prop = ctx['water-flow'](name='tank_outlet_water_flow')
# flow_prop['name-owner'] = my_tank['out']
# # now create the sensor
# flow_sensor = ctx['sensor'](name='my_flow_sensor')
# flow_sensor['where'] = my_tank['out'] # the sensor measures at the outlet of the tank
# flow_sensor['property'] = flow_prop['name'] # associate the sensor's "property" param with the flow property

In [17]:
# bldg_config
# assuming cooling+only and reheat vavs split evenly. Computer spaces are all in the cooling only vavs.
# [ahus, cooling_only_vavs_per_ahu, reheat_vavs_per_ahu]
# Could add amt computer spaces_per_ahu to building if it serves example later

# the context helps us build up our model
ctx = TemplateBuilderContext(BLDG)
ctx.add_templates_from_library(brick)
ctx.add_templates_from_library(brick_tmpl)

In [18]:
bldg_config = {'ahus': 2, 'cv': 10, 'hv': 10}
bldg_ctx_dict = {}
for i in range(bldg_config['ahus']):
    ahu_name = f'ahu_{i}'
    # bldg_ctx_dict[name] = ctx['multiple-zone-ahu'](name=name)
    ahu = ctx['multiple-zone-ahu'](name=ahu_name)
    for cv in range(bldg_config['cv']):
        cv_name = f'cvav_{i}_{cv}'
        cvav = ctx['vav-cooling-only'](name=cv_name)
        ctx['feeds'](name = ahu['name'], target = cvav['name'])
    for hv in range(bldg_config['hv']):
        hv_name = f'hvav_{i}_{hv}'
        hvav = ctx['vav-with-reheat'](name=hv_name)
        ctx['feeds'](name = ahu['name'], target = hvav['name'])

In [19]:
bldg.add_graph(ctx.compile())

In [20]:
print(bldg.graph.serialize(format='ttl'))

@prefix : <urn:example#> .
@prefix brick: <https://brickschema.org/schema/Brick#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

: a owl:Ontology .

:ahu_0 a brick:AHU ;
    rdfs:label "AHU" ;
    brick:feeds :cvav_0_0,
        :cvav_0_1,
        :cvav_0_2,
        :cvav_0_3,
        :cvav_0_4,
        :cvav_0_5,
        :cvav_0_6,
        :cvav_0_7,
        :cvav_0_8,
        :cvav_0_9,
        :hvav_0_0,
        :hvav_0_1,
        :hvav_0_2,
        :hvav_0_3,
        :hvav_0_4,
        :hvav_0_5,
        :hvav_0_6,
        :hvav_0_7,
        :hvav_0_8,
        :hvav_0_9 ;
    brick:hasPart :clg_coil_292e8e45,
        :clg_coil_570f24d9,
        :clg_coil_89d14339,
        :clg_coil_fcef1c18,
        :htg_coil_1277ef1a,
        :htg_coil_383e832f,
        :htg_coil_57af4df1,
        :htg_coil_d79c0b40,
        :ra_damper_49914130,
        :ra_damper_5f810f49,
        :ra_damper_cc533f45,
        :ra_damper_fcc813d4,
        

In [21]:
bldg.graph.serialize('brick-example.ttl', format = 'ttl')

<Graph identifier=09e2cf4c-78b1-4db4-8ddd-d5593d34ff29 (<class 'rdflib.graph.Graph'>)>